## 2.11. 모델 평가

모델 평가는 학습 모델의 성능을 측정하고 개선하기 위해서 필수적으로 필요한 과정이다. (공부를 했으면 시험을 봐야지 ... )

**대표적인 평가 방법론**

* **홀드아웃**: 데이터를 학습용과 테스트용으로 나누어 모델을 평가하는 방법 (여기저기서 많이 본 그 것)
* **교차검증**: 데이터를 여러 개의 폴드(Fold)로 나누어 각 폴드마다 학습과 검증을 반복해 모델 평가하는 방법
* **부트스트랩**: 데이터를 여러 번 샘플링해 각 샘플마다 모델을 학습하고 평가하는 방법

모델 평가에는 ***정량적 평가 지표***와 ***정서적 평가 지표***가 있다. 정량적 평가 지표는 모델의 성능을 수치로 나타낸 것이고, 정성적 평가 지표는 모델의 성능을 수치로 나타내기 어려운 것들이다. (말걸기?)

**분류 지표**

* **정확도(Accuracy)**: 전체 예측 데이터 건수 중 예측 결과가 일치하는 건수의 비율
* **정밀도(Precision)**: Positive로 예측한 데이터 건수 중 실제 Positive 건수의 비율
* **재현율(Recall)**: 실제 Positive 데이터 건수 중 Positive로 예측한 건수의 비율
* **F1 점수**: 정밀도와 재현율의 조화 평균
* **ROC-AUC**: 수신자 조작 특성 곡선 아래 면적

**회귀 지표**

* **평균 제곱근 오차(RMSE)**: 예측값과 실제값의 차이를 제곱해 평균한 값의 제곱근
* **평균 절대 오차(MAE)**: 예측값과 실제값의 차이를 절대값으로 변환해 평균한 값

**순위 지표**

* **평균 역순위(MRR)**: 첫 번째로 맞힌 정답의 역순위의 평균
* **K번째 정밀도(P@K)**: 상위 K개의 예측 결과 중 정답의 비율
* **감쇠누적 이득(DCG)**: 정답의 순위에 따라 점수를 부여해 더한 값

**자연어 처리 지표**

* **BLEU**: 기계 번역 결과와 실제 번역문 사이의 유사도를 측정하는 지표
* **ROUGE**: 요약 결과와 실제 요약문 사이의 유사도를 측정하는 지표
* **METEOR**: 기계 번역 결과와 실제 번역문 사이의 유사도를 측정하는 지표

**컴퓨터비전 처리 지표**

* **교차/합 비율(IoU)**: 객체 검출 모델의 예측 박스와 실제 박스 간의 겹침 정도
* **평균 정밀도(mAP)**: 객체 검출 모델의 정밀도 평균 값

평가 지표는 과제의 특성과 목적에 맞게 선택해야 한다. 예를 들어, 정확도는 데이터가 균일하게 분포되어 있을 때 사용하고, 정밀도와 재현율은 데이터가 불균일하게 분포되어 있을 때 사용한다. 또한, 평가 지표는 하나만 사용하는 것이 아니라 여러 개를 함께 사용하는 것이 좋다.




### 2.11.1. 평가 라이브러리

허깅페이스는 평가(evaluate) 라이브러리는 머신러닝 및 딥러닝 모델의 성능을 평가하기 위해 사용되는 도구다. 이 라이브러리는 다양한 평가 지표를 제공하며, 다양한 데이터셋에 대한 평가를 쉽게 수행할 수 있다.

```python
metric = evalute.load(...)
```
**load함수 인자**

* **path**: 평가 모듈의 경로다. 로컬 경로 또는 허깅페이스 평가 리포지토리의 모듈 식별자를 입력한다.
* **config_name**: 평가 지표의 구성을 선택한다. GLUE벤치 마크의 경우 MNLI, QQP, QNLI등 여러 하위작업이 있다 이 매개변수를 사용하면 특정 구성을 선택할 수 있다.
* **module_type**: 평가 모듈 유형으로 metric, comparison, measurement 중 하나를 할당한다. 
  * **평가지표(metric)**: 모델 성능을 평가한다. 모델 예측과 실제 레이블이 필요하며 정확도(Accuracy), 정밀도(Precision), 재현율(Recall) 등의 평가 지표를 제공한다.
  * **비교(comparison)**: 두 모델의 성능을 비교한다. 두 모델의 예측 결과가 필요하며, 두 모델의 성능 차이를 측정한다.
  * **측정(measurement)**: 모델의 성능을 측정한다. 모델 예측과 실제 레이블이 필요하며, 모델의 성능을 측정한다.

In [7]:
# 이미지 모델 평가 수행

import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification

dataset = load_dataset("cifar10", split="test[:100]")
model_name = "nateraw/vit-base-patch16-224-cifar10"
model = AutoModelForImageClassification.from_pretrained(model_name)
image_processor = AutoImageProcessor.from_pretrained(model_name)

model.eval()

batch_size = 32

logits, labels = [], []

for i in range(0, len(dataset), batch_size):
  batch = dataset[i : i + batch_size]
  with torch.no_grad():
    inputs = image_processor(batch["img"], return_tensors="pt")
    outputs = model(inputs["pixel_values"])

  logits.extend(outputs.logits.cpu().numpy())
  labels.extend(batch["label"])

preds = np.argmax(logits, axis=-1)

metric = evaluate.load("accuracy")
accuracy = metric.compute(predictions=preds, references=labels)
print(accuracy)

metric = evaluate.load("f1")
f1 = metric.compute(predictions=preds, references=labels, average="macro")
print(f1)

comparison = evaluate.load("exact_match", module_type="comparison")
exact_match = comparison.compute(predictions1=preds, predictions2=labels)
print(exact_match)

measurement = evaluate.load("label_distribution", module_type="measurement")
distribution = measurement.compute(data=labels)
print(distribution)


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


{'accuracy': 0.99}
{'f1': 0.9888544891640867}


{'exact_match': np.float64(0.99)}


{'label_distribution': {'labels': [3, 8, 0, 6, 1, 9, 5, 7, 4, 2], 'fractions': [0.1, 0.13, 0.1, 0.16, 0.06, 0.11, 0.08, 0.11, 0.07, 0.08]}, 'label_skew': np.float64(-0.2906781375329873)}
